### Decission Trees

Cuya traducción en español sería, árboles de decisión. 

Este algoritmo al igual que los SVM, puede realizar las tareas de regresión y clasificación con incluso múltiples salidas, también es parte fundamental del algoritmo Random Forest el cual es uno de los algoritmos más poderosos actualmente.

Ahora veremos como entrenar, visualizar y realizar predicciones con los árboles de decisiones, llamaremos a las tareas de clasificación y regresión para los árboles como CART (CART : Clasificación and Regression Trees). Usaremos el algoritmo CART de entrenamiento dado por la librería scikit-learn, regularizaremos los árboles y los usaremos para tareas de regresión. Finalmente se verá también las debilidades del algoritmo Decission Tree.

In [7]:
#### Setup
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

### Entrenamiento y Visualización de un árbol de decisión

Para entender los árboles de decisión se construirá uno y se mirará como se realiza una predicción.

In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [9]:
iris = load_iris()

X = iris.data[:, 2:] # largo y ancho del pétalo
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

#### Visualization

In [12]:
from sklearn.tree import export_graphviz
export_graphviz(
    tree_clf,
    out_file="/home/nicerova/Escritorio/iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

![](iris_tree.png)

### Haciendo predicciones

Veamos como es que trabaja un árbol de decisión:
   
Tenemos un nodo raiz en el cual tiene una "pregunta" y es que si el largo del pétalo es menor igual a 2.45 entonces en base a la respuesta de ésta pregunta es que podemos clasificar una flor, en este caso si la respuesta es SI entonces clasificado como setosa (la hoja izquierda).

Si la respuesta es FALSE nos movemos hacia la derecha donde el nodo raiz tiene un nodo hijo que de nuevo nos hace una pregunta que esta vez es **¿el pétalo tiene ancho = 1.75?** formandose en base a la respuesta SI o NO (TRUE o FALSE) dos hojas (nodos sin hijos) las cuales clasificará el valor como versicolor o como virginica según sea la respuesta.

Esto es realmente asi de simple.

#### Tip:

Uno de las muchas cualidades del algoritmo árbol de decisión es que requiere muy poca preparación de data, de hecho este algoritmo no requiere escalado de los datos o un centrado de estos.

#### Descripción del árbol

En la imagen podemos ver la opción *samples* la cual es igual a la cantidad de valores de entrenamiento que se encuentran en esa clasificación: Podemos ver como en el nodo raiz se encuentran los 150 valores de entrenamiento inicial, luego en la hoja izquierda tenemos 50 plantas las cuales la longitud de su pétalo es menor o igual a 2.45cm y por el lado derecho tenemos los 100 valores restantes, luego de estos 100 en sus nodos hijos izquierdos y derechos tenemos 54 (que son la cantidad de plantas cuyo ancho de su pétalo es menor igual a 1.75cm) y 46 valores respectivamente.

El atributo *value* en cada nodo indica cuantos de estos valores totales (vistos en el anterior párrafo) corresponden a cada clase: por ejemplo, tenemos en el nodo hijo más a la izquierda a 50 valores todos correspondientes a una única clase, la clase **setosa** por otro lado en el nodo color verde claro, 0 son **setosa**, 49 son **versicolor** y 5 son **virginica**.

Finalmente, el atributo *gini* indica la cantidad de impureza de un nodo, un gini = 0 significa un nodo "puro" el cual todos los valores que ahí se ubican (valores de entrenamiento) son de la misma clase que indica la hoja. Por ejemplo podemos encontrar este nivel de pureza (Gini = 0) en la hoja que está más a la izquierda ya que los valores de entrenamiento todos son setosa y justamente esta hoja indica que todos ahi que todos los valores que caigan serán setosas. Veamos como es la ecuación con la cual calculamos el atributo *Gini* el cual es llamado aquí como puntaje gini (Gini score).

$G_i = 1 - \sum_{k=1}^n p_{i,k^2}$

Donde:
   * i es el i-ésimo nodo
   * $p_{i,k}$ es la proporción de instancias de clase k entre las instancias de entrenamiento del i-ésimo nodo.
   
Por ejemplo, para la el hijo izquierdo del nodo derecho de la raiz tenemos :

$1 - ( (0/54)^2  + (49/54)^2 + (5/54)^2 ) \approx 0.168$

#### Tip 2: 

Este algoritmo CART que tiene la librería scikit-learn solo produce árboles binarios, es decir, los nodos que no son hojas tienen siempre dos hijos (preguntas si o no) , sin embargo otros algoritmos como ID3 pueden producir árboles de decisión con nodos que tiene más de dos hijos.

### Caja blanca, caja negra

Como se vió es bastante simple la explicación de la clasificación que hizo nuestro árbol de decisión, incluso hasta se pudo graficar las preguntas que se hizo para clasificar nuestro valores, a este compartamiento se le llama modelos de caja blanca. Por otro lado, en el algoritmo RandomForest o en las redes Neuronales es muy complicado decir como es que se hizo las preguntas, como es que se tuvo la clasificación de cierta clase y todas estas preguntas que se podian responder en los modelos de caja blanca. Por ejemplo, si una red neuronal identifica una persona en una foto, no podemos nosotros explicar con certeza que parámetros de la foto influyó en la clasificación de la persona, pudo haber sido sus ojos, su boca, su rostro en general, esto no es del todo claro como en los modelos de caja blanca (hasta los podemos hacerlos manualmente !!!)

### Estimación de probabilidades de las clases

Un árbol de decisión también puede estimar la probabilidad de que una instancia pertenezca a una clase k particular: primero atraviesa el árbol para encontrar el nodo hoja para esta instancia, y luego devuelve la proporción de instancias de entrenamiento de la clase k en este nodo. 

Por ejemplo, supongamos que hemos encontrado una flor cuyos pétalos son de 5 cm de largo y 1,5 cm de ancho. El nodo hoja correspondiente es el nodo izquierdo de profundidad 2, por lo que el Árbol de decisión debería generar las siguientes probabilidades: 0% para Iris-Setosa (0/54), 90.7% para Iris-Versicolor (49/54) y 9.3% para Iris-Virginica (5/54). Y, por supuesto, si nos piden que se prediga la clase, se debería generar Iris-Versicolor (clase 1) ya que tiene la mayor probabilidad.

In [19]:
tree_clf.predict_proba([[5, 1.5]]) # las 3 clases con sus respectivas probabilidades

array([[0.        , 0.90740741, 0.09259259]])

In [21]:
tree_clf.predict([[5, 1.5]])

array([1])